<a href="https://colab.research.google.com/github/EmmaMills1002/DS2002Project1/blob/main/CSVtoJSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Import pandas
import pandas as pd

# Load the csv file into a data frame
sabrina_df = pd.read_csv('/content/sabrina_carpenter_discography.csv', delimiter=';')

# Create new column 'dance_party' where the value is True if danceability is greater than 0.75
sabrina_df['dance_party'] = sabrina_df['danceability'] > 0.75

# Look at first few rows of the data
sabrina_df.head()

,track_id,track_name,track_type,track_musical_genre,duration_ms,spotify_streams,spotify_global_peak,spotify_usa_peak,track_videoclip,videoclip_views,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,dance_party
0,0,Taste,Single,Rock Pop,157280,225.991.075,3,1,Yes,72.175.544,...,0.101,0.674,0.907,0,0.2970,-4.086,0.0640,112.964,0.721,False
1,1,Please Please Please,Single,Country Pop,186365,799.883.111,1,1,Yes,110.879.296,...,0.257,0.675,0.586,0,0.1040,-6.086,0.0531,107.015,0.621,False
2,2,Good Graces,B-Side,R&B,185265,84.670.729,11,7,No,0,...,0.329,0.758,0.759,3.96e-06,0.1440,-5.315,0.0479,126.012,0.732,True
3,3,Sharpest Tool,B-Side,Folk Pop,218284,55.698.940,25,12,No,0,...,0.475,0.716,0.630,0,0.0943,-7.509,0.1080,91.003,0.588,False
4,4,Coincidence,B-Side,Country Pop,164203,57.407.951,28,15,No,0,...,0.362,0.418,0.815,0,0.1250,-6.216,0.1250,77.862,0.758,False


In [27]:

# Number of columns
num_columns = len(sabrina_df.columns)
print(f"Number of columns: {num_columns}")

# Number of records (rows)
num_records = len(sabrina_df)
print(f"Number of records: {num_records}")

Number of columns: 26
Number of records: 68


In [9]:
import json

# Convert the csv to a json file

sabrina_json = sabrina_df.to_json(orient='records')
sabrina_data = json.loads(sabrina_json)

# Look at first observation in json file

sabrina_data[0]

{'track_id': 0,
 'track_name': 'Taste',
 'track_type': 'Single',
 'track_musical_genre': 'Rock Pop',
 'duration_ms': 157280,
 'spotify_streams': '225.991.075',
 'spotify_global_peak': 3,
 'spotify_usa_peak': 1,
 'track_videoclip': 'Yes',
 'videoclip_views': '72.175.544',
 'album': "Short n' Sweet",
 'album_physical_sales': 342500,
 'track_number': 1,
 'album_musical_genre': 'Pop',
 'release_date': '23/8/2024',
 'uri': 'spotify:track:5G2f63n7IPVPPjfNIGih7Q',
 'acousticness': 0.101,
 'danceability': 0.674,
 'energy': 0.907,
 'instrumentalness': '0',
 'liveness': 0.297,
 'loudness': -4.086,
 'speechiness': 0.064,
 'tempo': 112.964,
 'valence': 0.721,
 'dance_party': False}

In [24]:
# Import sqlite3 and create sabrina_discography SQL database
import sqlite3
conn = sqlite3.connect('sabrina_discography.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS sabrina (
  track_id TEXT PRIMARY KEY,
  track_name TEXT,
  track_type TEXT,
  track_musical_genre TEXT,
  duration_ms INTEGER,
  spotify_streams INTEGER,
  spotify_global_peak INTEGER,
  spotify_usa_peak INTEGER,
  track_videoclip TEXT,
  videoclip_views REAL,
  album TEXT,
  album_physical_sales TEXT,
  track_number INTEGER,
  album_musical_genre TEXT,
  release_date TEXT,
  uri TEXT,
  acousticness REAL,
  danceability REAL,
  energy REAL,
  instrumentalness REAL,
  liveness REAL,
  loudness REAL,
  speechiness REAL,
  tempo REAL,
  valence REAL,
  dance_party TEXT
)
''')
for index, row in sabrina_df.iterrows():
    columns = ', '.join(row.keys())
    placeholders = ', '.join(['?'] * len(row))
    sql = f'INSERT OR IGNORE INTO sabrina ({columns}) VALUES ({placeholders})'
    cursor.execute(sql, tuple(row))

    conn.commit()


In [25]:
# Use SQL database to output Sabrina tracks and their albums in order of spotify streams from most to least

query1 = '''
SELECT
sabrina.track_name,
sabrina.album
FROM
sabrina
ORDER BY
sabrina.spotify_streams DESC
LIMIT 10
'''

cursor.execute(query1)
results = cursor.fetchall()
for row in results:
  print(row)

('Nonsense', 'emails i can’t send')
('Good Graces', "Short n' Sweet")
('Two Young Hearts', 'Eyes Wide Open')
("Don't Want it Back", 'EVOLution')
('White Flag', 'Eyes Wide Open')
('Please Please Please', "Short n' Sweet")
('Fast Times', 'emails i can’t send')
('Hold Tight', 'Singular: Act I')
('No Words', 'EVOLution')
('Mirage', 'EVOLution')


In [26]:

# Column count - check to see if the same as original CSV
cursor.execute("SELECT COUNT(*) FROM pragma_table_info('sabrina')")
column_count = cursor.fetchone()[0]
print(f"Number of columns: {column_count}")

# Record count - check to see if the same as original CSV
cursor.execute("SELECT COUNT(*) FROM sabrina")
record_count = cursor.fetchone()[0]
print(f"Number of records: {record_count}")

# Close the connection
conn.close()


Number of columns: 26
Number of records: 68
